In [1]:
cat > Dockerfile <<EOF
FROM ubuntu:18.04
RUN apt-get update && apt-get install -y software-properties-common && add-apt-repository -y ppa:alex-p/tesseract-ocr
RUN apt-get update && apt-get install -y tesseract-ocr-all 
RUN mkdir /work
WORKDIR /work
EOF

In [2]:
sudo apt-get install -y dtrx rename

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-container-tools libnvidia-container1 libssl-doc
  nvidia-container-runtime nvidia-container-runtime-hook
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  cabextract debugedit librpm8 librpmbuild8 librpmio8 librpmsign8 libunshield0
  p7zip p7zip-full rpm rpm-common rpm2cpio unshield
Suggested packages:
  rpm-i18n p7zip-rar alien rpmlint rpm2html
The following NEW packages will be installed:
  cabextract debugedit dtrx librpm8 librpmbuild8 librpmio8 librpmsign8
  libunshield0 p7zip p7zip-full rename rpm rpm-common rpm2cpio unshield
0 upgraded, 15 newly installed, 0 to remove and 0 not upgraded.
Need to get 2111 kB of archives.
After this operation, 8027 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 c

In [36]:
apt-cache search libtiff

libtiff-dev - Tag Image File Format library (TIFF), development files, current version
libtiff-doc - TIFF manipulation and conversion documentation
libtiff5 - Tag Image File Format (TIFF) library
libtiff5-dev - Tag Image File Format library (TIFF), development files
libtiffxx5 - Tag Image File Format (TIFF) library -- C++ interface
gem-plugin-tiff - Graphics Environment for Multimedia - TIFF support
libtiff-opengl - TIFF manipulation and conversion tools
libtiff-tools - TIFF manipulation and conversion tools
pngtools - series of tools for PNG (Portable Network Graphics) images


In [37]:
sudo apt-get install -y graphicsmagick libtiff5-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
graphicsmagick is already the newest version (1.3.28-2).
The following packages were automatically installed and are no longer required:
  libnvidia-container-tools libnvidia-container1 libssl-doc
  nvidia-container-runtime nvidia-container-runtime-hook
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  libjbig-dev libjpeg-dev libjpeg-turbo8-dev libjpeg8-dev liblzma-dev
  libtiffxx5
Suggested packages:
  liblzma-doc
The following NEW packages will be installed:
  libjbig-dev libjpeg-dev libjpeg-turbo8-dev libjpeg8-dev liblzma-dev
  libtiff5-dev libtiffxx5
0 upgraded, 7 newly installed, 0 to remove and 0 not upgraded.
Need to get 678 kB of archives.
After this operation, 2645 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-security/main amd64 libjpeg-turbo8-dev amd64 1.5.2-0ubuntu5.18.04.1 [225 kB]
Get:2 htt

In [3]:
docker build -t tess - < Dockerfile

Sending build context to Docker daemon  2.048kB
Step 1/5 : FROM ubuntu:18.04
 ---> 1d9c17228a9e
Step 2/5 : RUN apt-get update && apt-get install -y software-properties-common && add-apt-repository -y ppa:alex-p/tesseract-ocr
 ---> Using cache
 ---> 7251a699f211
Step 3/5 : RUN apt-get update && apt-get install -y tesseract-ocr-all
 ---> Using cache
 ---> 7e9a74a42554
Step 4/5 : RUN mkdir /work
 ---> Using cache
 ---> a5f0eda64a83
Step 5/5 : WORKDIR /work
 ---> Using cache
 ---> 46adca2436b0
Successfully built 46adca2436b0
Successfully tagged tess:latest


In [4]:
cat > run-tess <<\EOF
#!/bin/bash
docker run -v $(pwd):/work -w /work -t tess tesseract "$@"
EOF
chmod 755 run-tess

In [5]:
run-tess --help

Usage:
  tesseract --help | --help-extra | --version
  tesseract --list-langs
  tesseract imagename outputbase [options...] [configfile...]

OCR options:
  -l LANG[+LANG]        Specify language(s) used for OCR.
NOTE: These options must occur before any configfile.

Single options:
  --help                Show this help message.
  --help-extra          Show extra help for advanced users.
  --version             Show version information.
  --list-langs          List available languages for tesseract engine.


In [9]:
gsutil ls gs://lpr-cdip | head

gs://lpr-cdip/README.rst
gs://lpr-cdip/images.a.a.cpio
gs://lpr-cdip/images.a.b.cpio
gs://lpr-cdip/images.a.c.cpio
gs://lpr-cdip/images.a.f.cpio
gs://lpr-cdip/images.a.g.cpio
gs://lpr-cdip/images.a.h.cpio
gs://lpr-cdip/images.a.i.cpio
gs://lpr-cdip/images.a.j.cpio
gs://lpr-cdip/images.a.k.cpio


In [19]:
volume=images.b.b
test -f volume.cpio || gsutil cp gs://lpr-cdip/images.b.b.cpio volume.cpio

Copying gs://lpr-cdip/images.b.b.cpio...
- [1 files][  1.9 GiB/  1.9 GiB]   42.7 MiB/s                                   
Operation completed over 1 objects/1.9 GiB.                                      


In [1]:
rm -rf volume
rm -rf outputs

In [2]:
dtrx --one rename volume.cpio

In [3]:
find volume -name '*.tif' | wc -l
find volume -name '*.xml' | wc -l

13763
10656


In [ ]:
xargs --help

In [26]:
rm -rf outputs
mkdir outputs
find volume -name '*.tif' |
while read f; do
    base=$(echo $f | sed 's!volume/!!;s!/!-!g;s/\.tif$//')
    #echo $f $base
    echo convert $f outputs/${base}-%04d.png
done | 
xargs -d\\n -i bash -c {}

convert-im6.q16: Unknown field with tag 292 (0x124) encountered. `TIFFReadDirectory' @ warning/tiff.c/TIFFWarnings/912.
convert-im6.q16: ASCII value for tag "Software" does not end in null byte. `TIFFFetchNormalTag' @ warning/tiff.c/TIFFWarnings/912.
convert-im6.q16: Unknown field with tag 292 (0x124) encountered. `TIFFReadDirectory' @ warning/tiff.c/TIFFWarnings/912.
convert-im6.q16: Unknown field with tag 292 (0x124) encountered. `TIFFReadDirectory' @ warning/tiff.c/TIFFWarnings/912.
convert-im6.q16: ASCII value for tag "Software" does not end in null byte. `TIFFFetchNormalTag' @ warning/tiff.c/TIFFWarnings/912.
convert-im6.q16: Unknown field with tag 292 (0x124) encountered. `TIFFReadDirectory' @ warning/tiff.c/TIFFWarnings/912.
convert-im6.q16: Invalid TIFF directory; tags are not sorted in ascending order. `TIFFReadDirectoryCheckOrder' @ warning/tiff.c/TIFFWarnings/912.
convert-im6.q16: Unknown field with tag 292 (0x124) encountered. `TIFFReadDirectory' @ warning/tiff.c/TIFFWarnin

: 123

In [ ]:
find outputs -name '*.png' | wc -l

In [ ]:
find outputs -name '*.png' | sed 5q

In [56]:
mkdir outputs
find volume -name '*.png' | 
while read f; do
    base=$(echo $f | sed 's/volume\///;s/\//-/g')
    echo $f $base
    break
done

mkdir: cannot create directory ‘outputs’: File exists
volume/b/b/z/bbz09e00/2049045975_%04d.png b-b-z-bbz09e00-2049045975_%04d.png
find: ‘standard output’: Broken pipe
find: write error
